In [165]:
import matplotlib.pyplot as plt
import xesmf as xe
import numpy as np
from netCDF4 import Dataset
import xarray as xr
import os
import iris

In [212]:
# prepare regridder

dummy = np.zeros([1,777602])
dummy_b = np.zeros([2,777603])
gridIn = {'lon': dummy, 'lon_b': dummy_b, 'lat': dummy, 'lat_b': dummy_b}
gridOut = xe.util.grid_2d(-180,180,0.5,-90,90,0.5)
regridder = xe.Regridder(gridIn, gridOut, 'conservative',reuse_weights=True,\
                         weights='/home/b/b381639/Analysis/conservative_1x777602_360x720.nc')



#readin data
mod = xr.open_mfdataset('/home/b/b381639/Analysis/SST_0150_12_*.nc')
datain=mod['SST']
time=datain.time
dt=datain.shape[0]


#prepare lat lon grid for output data with a original data with one timestep
latlonin=xr.open_dataset('/home/b/b381639/Analysis/SST_0150_12_01.nc')['SST']
latlon=regridder(latlonin)
lon=latlon.lon[0,:]
lat=latlon.lat[:,0]
s=latlon.shape
zero=np.zeros((dt, s[0],s[1])) # data shape of output data

latitude=xr.IndexVariable(data=lat,dims='latitude',attrs={'units':'degrees_north'})
longitude=xr.IndexVariable(data=lon,dims='longitude',attrs={'units':'degrees_east'})
Dsout=xr.Dataset(coords={'time':time[0],'latitude':latitude,'longitude':longitude})
Dsout['SST']=xr.DataArray(data=np.copy(zero),coords={"time":time,'latitude':latitude,'longitude':longitude},dims=['time','latitude','longitude'],attrs={'units':'K','long_name': 'sea surface temperature'})



s=datain.shape
redata=np.zeros((1,s[1]))
for i in range(0,dt):
    redata[0,:]=datain[i,:]
    Dsout.SST.data[i,:,:]=regridder(redata)

dir1="/home/b/b381639/Analysis/"
os.system('rm '+dir1+'test.nc')
Dsout.to_netcdf(dir1+'test.nc')

/sw/spack-rhel6/miniforge3-4.9.2-3-Linux-x86_64-pwdbqi/lib/python3.8/site-packages/xesmf/frontend.py:439: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


In [166]:
###varin is imput data to be regridded. the shape should be 77602. lon lat is generated with the same regridder 
def regress_latlon(varin, lon,lat):
    
    s=datain.shape
    redata=np.zeros((1,s[0]))
    redata[0,:]=datain

    ur=regridder(varin)
    s=ur.shape
    zero=np.zeros((1,s[0],s[1]))

    latitude=xr.IndexVariable(data=lat,dims='latitude',attrs={'units':'degrees_north'})
    longitude=xr.IndexVariable(data=lon,dims='longitude',attrs={'units':'degrees_east'})

    time=varin.time

    Dsout=xr.Dataset(coords={'time':time,'latitude':latitude,'longitude':longitude})
    Dsout['SST']=xr.DataArray(data=np.copy(zero),coords={"time":time,'latitude':latitude,'longitude':longitude}, \
                          dims=['time','latitude','longitude'], \
                          attrs={'units':'K','long_name': 'sea surface temperature'})

    return(Dsout)

In [188]:
    time

<xarray.DataArray 'time' (time: 9)>
array([cftime.DatetimeNoLeap(150, 12, 1, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 2, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 3, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 4, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 5, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 6, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 7, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 8, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(150, 12, 9, 0, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 0150-12-01 00:00:00 ... 0150-12-09 00:00:00
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T

In [189]:
    Dsout=xr.Dataset(coords={'time':time[0],'latitude':latitude,'longitude':longitude})

()

In [190]:
    Dsout['SST']=xr.DataArray(data=np.copy(zero),coords={"time":time[0],'latitude':latitude,'longitude':longitude},dims=['time','latitude','longitude'],attrs={'units':'K','long_name': 'sea surface temperature'})

ValueError: coordinate 'time' is a DataArray dimension, but it has shape () rather than expected shape 1 matching the dimension size

In [118]:
    Dsout=xr.Dataset(coords={'time':time,'latitude':latitude,'longitude':longitude})
    Dsout['SST']=xr.DataArray(data=np.copy(zero),coords={"time":time,'latitude':latitude,'longitude':longitude}, \
                          dims=['time','latitude','longitude'], \
                          attrs={'units':'K','long_name': 'sea surface temperature'})

ValueError: coordinate 'time' is a DataArray dimension, but it has shape () rather than expected shape 1 matching the dimension size

In [79]:
s=datain.shape
redata=np.zeros((1,s[0]))
redata.shape

(1, 777602)

<ipython-input-104-c4c55c913b94>:1: DeprecationWarning: Assigning the 'data' attribute is an inherently unsafe operation and will be removed in the future.
  redata[0,:].data=datain


TypeError: a bytes-like object is required, not 'DataArray'

In [86]:
datain.time

<xarray.DataArray 'time' ()>
array(cftime.DatetimeNoLeap(150, 12, 1, 0, 0, 0, 0), dtype=object)
Coordinates:
    time     object 0150-12-01 00:00:00
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T

In [82]:
redata.shape

(1, 777602)

In [83]:
ur=regridder(redata)

In [85]:
ur.shape

(360, 720)

AttributeError: 'numpy.ndarray' object has no attribute 'lon'

In [49]:



year_cubelist = iris.cube.CubeList()

In [57]:
dirout="/home/b/b381639/Analysis/"
for d in range(0,8):

    cube=regress_latlon(varin,dirout)

#    year_cubelist.append(cube)

AttributeError: 'numpy.ndarray' object has no attribute 'lon'

In [56]:
varin.shape

(1, 777602)

In [39]:


datain[8,:].shape




(777602,)

In [28]:
dirout="/home/b/b381639/Analysis/"
for d in range(1,9):
    
    
    
    for m in range(12,13):
        if m==2:
            for d in range(1,29):
                fname=dir+"SST_"+str(y).zfill(4)+"_"+str(m).zfill(2)+"_"+str(d).zfill(2)+".nc"
                print(fname)
                #Ds=regress_latlon(fname,dirout,y,m,d)
        elif m in (1,3,5,7,8,10,12):        
            for d in range(1,32):
                fname=dir+"SST_"+str(y).zfill(4)+"_"+str(m).zfill(2)+"_"+str(d).zfill(2)+".nc"
                print(fname)
                #Ds=regress_latlon(fname,dirout,y,m,d)
        else:
             for d in range(1,31):
                fname=dir+"SST_"+str(y).zfill(4)+"_"+str(m).zfill(2)+"_"+str(d).zfill(2)+".nc"       
                print(fname)
                #Ds=regress_latlon(fname,dirout,y,m,d)
      
                
            
            
            

TypeError: unsupported operand type(s) for +: 'builtin_function_or_method' and 'str'

In [29]:
Ds=regress_latlon(fname,dirout,y,m,d)

NameError: name 'fname' is not defined

In [30]:
dir1="/home/b/b381639/Analysis/"
os.system('rm '+dir1+'test.nc')
Dsout.to_netcdf(dir1+'test.nc')

NameError: name 'Dsout' is not defined

In [ ]:
def regress_latlon(fnamein):
    fin=xr.open_dataset(fnamein)
    U=fin.SST
    ur=regridder(U)
    s=ur.shape
    zero=np.zeros((1,s[0],s[1]))
    
    lon=ur.lon[0,:]
    lat=ur.lat[:,0]
    latitude=xr.IndexVariable(data=lat,dims='latitude',attrs={'units':'degrees_north'})
    longitude=xr.IndexVariable(data=lon,dims='longitude',attrs={'units':'degrees_east'})

    time=U.time

    Dsout=xr.Dataset(coords={'time':time,'latitude':latitude,'longitude':longitude})
    Dsout['SST']=xr.DataArray(data=np.copy(zero),coords={"time":time,'latitude':latitude,'longitude':longitude}, \
                          dims=['time','latitude','longitude'], \
                          attrs={'units':'K','long_name': 'sea surface temperature'})

    Dsout.SST.data[0,:,:]=ur

    return(Dsout)
dir1="/home/b/b381639/Analysis/"
os.system('rm '+dir1+'test.nc')
Dsout.to_netcdf(dir1+'test.nc')

In [ ]:
#################################
#################
Year=range(1,39)
latitude=xr.IndexVariable(data=lat,dims='latitude',attrs={'units':'degrees_north'})
longitude=xr.IndexVariable(data=lon,dims='longitude',attrs={'units':'degrees_east'})

zero=np.zeros((38,360,720))
MC=xr.Dataset(coords={'time':time,'latitude':latitude,'longitude':longitude})        
MC['MC1']=xr.DataArray(data=np.copy(zero),coords={"Year":Year,'latitude':latitude,'longitude':longitude}, dims=['Year','latitude','longitude'], attrs={'units':'mm day**-1'})

MC.MC1.data=Var1



dir1="/home/b/b381639/Analysis/"
###########################################


os.system('rm '+dir1+'MC_mmd_eot.nc')
MC.to_netcdf(dir1+'MC_mmd_eot.nc')